## Lab1 - Customer Churn (Marketing)

In [206]:
import boto3
import json
import pandas as pd

sm_client = boto3.client('sagemaker')
runtime_client = boto3.client("runtime.sagemaker")

In [208]:
# Configuration
endpoint_name = "canvas-new-deployment-02-12-2024-4-07-PM"  ## Replace Endpoint name
variant_name = sm_client.describe_endpoint(EndpointName=endpoint_name)['ProductionVariants'][0]['VariantName']

### Test data

In [209]:
body = "MO,176,777,913-2699,no,yes,400,1.639329295774005,3,3.678888958431078,3.0296212421701054,1,3.419607317320714,3.1829204913988565,150,3.701859799002945,4.4130370771175285,6,2.728670064271372,7"

### boto3 방식 invocation

In [294]:
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="text/csv",
    Body=body,
    Accept="application/json"
)
data = response["Body"].read().decode("utf-8")
output = json.loads(data)["predictions"]
output

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "Invalid data format. Input data has 20 while the model expects 8 ['UDI', 'Product ID', 'Type', 'Air temperature', 'Process temperature', 'Rotational speed', 'Torque', 'Tool wear'] <class 'list'>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sagemaker_inference/transformer.py", line 128, in transform
    result = self._transform_fn(self._model, input_data, content_type, accept)
  File "/opt/ml/model/code/tabular_serve.py", line 80, in transform_fn
    raise Exception(
Exception: Invalid data format. Input data has 20 while the model expects 8 ['UDI', 'Product ID', 'Type', 'Air temperature', 'Process temperature', 'Rotational speed', 'Torque', 'Tool wear'] <class 'list'>
". See https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/canvas-new-deploym

### SageMaker 방식 invocation

In [193]:
import sagemaker

# SageMaker configuration setup
session = sagemaker.session.Session()
serializer = sagemaker.serializers.CSVSerializer()
deserializer = sagemaker.deserializers.CSVDeserializer()

# Connect to the endpoint and query
predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name, 
    sagemaker_session=session,
    serializer=serializer,
    deserializer=deserializer
)
predictor.predict(body)


[['False.',
  '0.9965066909790039',
  '[0.9965066909790039, 0.003493297379463911]',
  "['False.', 'True.']"]]

## Chalice 를 이용한 Rest API 구성

In [194]:
!pip install chalice
!sudo yum -y install tree

Loaded plugins: dkms-build-requires, extras_suggestions, kernel-livepatch,
              : langpacks, priorities, update-motd, versionlock
amzn2-core                                               | 3.6 kB     00:00     
https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.
neuron                                                   | 2.9 kB     00:00     
62 packages excluded due to repository priority protections
Package tree-1.6.0-10.amzn2.0.1.x86_64 already installed and latest version
Nothing to do


In [247]:
from pathlib import Path
import jinja2
jinja_env = jinja2.Environment()

In [248]:
project_name = 'churn-prediction'
!rm -rf $project_name
!chalice new-project $project_name

Your project has been generated in ./churn-prediction


In [249]:
!tree -a $project_name

churn-prediction
├── app.py
├── .chalice
│   └── config.json
├── .gitignore
└── requirements.txt

1 directory, 4 files


#### Setup config.json
 > Chalice는 IAM 정책 자동 생성 기능이 있지만, 필요한 정책을 가진 IAM 정책을 생성할수 있습니다. 기본적으로는 직접 IAM 정책을 생성하는 것이 안전합니다.
 > 자세한 내용은 https://chalice-fei.readthedocs.io/en/latest/topics/configfile.html 를 참조하기 바랍니다.

##### autogen_policy:

- 애플리케이션 소스 코드 분석을 기반으로 chalice가 IAM 정책을 자동으로 생성할지 여부를 설정 (디폴트 = True)
- False인 경우, .chalice/policy-<단계 이름>.json에서 IAM 정책을 로드
- iam_policy_file 지정으로 불러올 policy 파일명을 변경할 수도 있음

In [250]:
%%writefile $project_name/.chalice/policy-dev.json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents",
                "logs:CreateLogGroup"
            ],
            "Resource": "arn:aws:logs:*:*:*"
        },
        {
            "Sid": "VisualEditor1",
            "Effect": "Allow",
            "Action": "sagemaker:InvokeEndpoint",
            "Resource": "*"
        }
    ]
}

Writing churn-prediction/.chalice/policy-dev.json


In [251]:
# %%writefile $project_name/.chalice/config.json

# {
#     "Version": "2.0",
#     "app_name": "churn-prediction",
#     "autogen_policy": false,
#     "automatic_layer": true,
#     "environment_variables": {
#         "ENDPOINT_NAME": "canvas-new-deployment-02-12-2024-4-07-PM"
#     },
#     "stages": {
#         "dev": {
#             "api_gateway_stage": "api",
#             "iam_policy_file": "policy-dev.json",
#             "subnet_ids": ["subnet-xxxxxxxxxxxxxx", "subnet-xxxxxxxxxxxxxx"],
#             "security_group_ids": ["sg-xxxxxxxxxxxxxx"],
#         }    
#     }
# }

In [252]:
config_filename = f'{project_name}/.chalice/config.json'

In [253]:
%%writefile $config_filename

{
    "Version": "2.0",
    "app_name": "churn-prediction",
    "autogen_policy": false,
    "automatic_layer": true,
    "environment_variables": {
        "ENDPOINT_NAME": "{{endpoint_name}}",
        "VARIANT_NAME" : "{{variant_name}}"
    },
    "stages": {
        "dev": {
            "api_gateway_stage": "api",
             "iam_policy_file": "policy-dev.json"
        }
    }
}

Overwriting churn-prediction/.chalice/config.json


In [254]:
template = jinja_env.from_string(Path(config_filename).open().read())
Path(config_filename).open("w").write(template.render(endpoint_name=endpoint_name, variant_name=variant_name))
!pygmentize $config_filename | cat -n

     1	{
     2	    "Version": "2.0",
     3	    "app_name": "churn-prediction",
     4	    "autogen_policy": false,
     5	    "automatic_layer": true,
     6	    "environment_variables": {
     7	        "ENDPOINT_NAME": "canvas-new-deployment-02-12-2024-4-07-PM",
     8	        "VARIANT_NAME" : "canvas-model-variant-2024-02-12-07-07-15-529579"
     9	    },
    10	    "stages": {
    11	        "dev": {
    12	            "api_gateway_stage": "api",
    13	             "iam_policy_file": "policy-dev.json"
    14	        }
    15	    }
    16	}


#### json test

In [255]:
json.loads('{"data":["MO,123,1"]}')

{'data': ['MO,123,1']}

In [256]:
%%writefile $project_name/app.py 
import os
import io
import json
import boto3
import base64
import numpy as np

from chalice import Chalice
from chalice import BadRequestError

app = Chalice(app_name="churn-prediction")
app.debug = True

runtime = boto3.client("runtime.sagemaker")

@app.route("/")
def index():
    return {'hello': 'world'}

@app.route("/variant/{variant_name}", methods=["POST"])
def return_prediction(variant_name):
    body = app.current_request.json_body
    
    if app._debug:
        print(f"body : {body} , \nvariant_name : {variant_name}")
        
    if "data" not in body:
        raise BadRequestError("Missing data.")
    if "ENDPOINT_NAME" not in os.environ:
        raise BadRequestError("Missing endpoint.")

    try:
        response = runtime.invoke_endpoint(
            EndpointName=os.environ["ENDPOINT_NAME"],
            TargetVariant=os.environ["VARIANT_NAME"],
            ContentType="text/csv",
            Body=body['data'],
            Accept="application/json"
        )
        data = response["Body"].read().decode("utf-8")
        output = json.loads(data)["predictions"]
        
        if app._debug:
            print(f"predictions : {output}")
        
        return {
            "response": {
                "value": json.dumps(output)
            }
        }
    except Exception as e:
        print(e)
        # print(payload)
    

Overwriting churn-prediction/app.py


In [257]:
%%writefile $project_name/requirements.txt
numpy

Overwriting churn-prediction/requirements.txt


### Local Test

In [267]:
api_url = 'localhost:8100/'

## Test data 로 Terminal에서 추가 테스트 수행
print(f"curl -X POST {api_url}variant/{variant_name} -H 'Content-Type: application/json'"+" -d '{\"data\":\""+body+"\"}'")

curl -X POST localhost:8100/variant/canvas-model-variant-2024-02-12-07-07-15-529579 -H 'Content-Type: application/json' -d '{"data":"MO,176,777,913-2699,no,yes,400,1.639329295774005,3,3.678888958431078,3.0296212421701054,1,3.419607317320714,3.1829204913988565,150,3.701859799002945,4.4130370771175285,6,2.728670064271372,7"}'


In [259]:
!cd $project_name && chalice local --port=8100

Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Restarting local dev server.
Found credentials in shared credentials file: ~/.aws/credentials
Serving on http://127.0.0.1:8100
body : {'data': 'MO,176,777,913-2699,no,yes,400,1.639329295774005,3,3.678888958431078,3.0296212421701054,1,3.419607317320714,3.1829204913988565,150,3.701859799002945,4.4130370771175285,6,2.728670064271372,7'} , 
variant_name : canvas-model-variant-2024-02-12-07-07-15-529579
predictions : [{'predicted_label': 'False.', 'probability': 0.9965066909790039, 'probabilities': '[0.9965

### Production Test

In [260]:
!cd $project_name && chalice deploy

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Creating shared layer deployment package.
Creating app deployment package.
Creating lambda layer: churn-prediction-dev-managed-layer
Creating IAM role: churn-prediction-dev-api_handler
Creating lambda function: churn-prediction-dev
Creating Rest API
Resources deployed:
  - Lambda Layer ARN: arn:aws:lambda:us-west-2:322537213286:layer:churn-prediction-dev-managed-layer:1
  - Lambda ARN: arn:aws:lambda:us-west-2:322537213286:function:churn-prediction-dev
  - Rest API URL: https://icbzqrl2y8.execute-api.us-west-2.amazonaws.com/api/


In [263]:
api_url = 'https://icbzqrl2y8.execute-api.us-west-2.amazonaws.com/api/'  ## chalice deploy의 결과 값에서 copy하여 변경

## Test data 로 Terminal에서 추가 테스트 수행
print(f"curl -X POST {api_url}variant/{variant_name} -H 'Content-Type: application/json'"+" -d '{\"data\":\""+body+"\"}'")

In [268]:
!cd $project_name && chalice delete

Deleting Rest API: icbzqrl2y8
Deleting function: arn:aws:lambda:us-west-2:322537213286:function:churn-prediction-dev
Deleting IAM role: churn-prediction-dev-api_handler
Deleting layer version: arn:aws:lambda:us-west-2:322537213286:layer:churn-prediction-dev-managed-layer:1


## Lab6

In [295]:
# Configuration
endpoint_name = "canvas-new-deployment-02-12-2024-8-45-PM"
variant_name = sm_client.describe_endpoint(EndpointName=endpoint_name)['ProductionVariants'][0]['VariantName']

In [296]:
body = pd.DataFrame(
    [['1', 'H29424', 'L', '297.4', '308.6', '1419', '35', '0']]
).to_csv(header=False, index=False).encode("utf-8") 

In [297]:
body

b'1,H29424,L,297.4,308.6,1419,35,0\n'

In [298]:
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    TargetVariant=variant_name,
    ContentType="text/csv",
    # ContentType="application/json",
    Body=body,
    Accept="application/json"
)
data = response["Body"].read().decode("utf-8")
output = json.loads(data)["predictions"]
output

[{'predicted_label': 'No Failure',
  'probability': 0.9645493626594543,
  'probabilities': '[0.9645493626594543, 0.0, 0.035450633615255356]',
  'labels': "['No Failure', 'Overstrain Failure', 'Power Failure']"}]